# Une analyse naïve des sentiments

Vous travaillez pour un voyagiste (*tour operator*) qui souhaite automatiser l’analyse des avis de ses clients. Pour mener à bien votre tâche, vous allez entraîner un classifieur bayésien naïf sur un jeu de données fictif afin de déterminer automatiquement, à partir d’un nouveau message, si l’avis exprimé est positif ou négatif.

Si les hypothèses derrière les modèles bayésiens naïfs sont très rarement vérifiées dans la réalité, ce type d’algorithme probabiliste montre malgré tout de très bons résultats. Au nombre de ses avantages, il ne nécessite en plus que très peu de données et les calculs requis sont relativement simples à implémenter.

## Présentation des ressources

Dans ce dossier, vous trouverez :

- un répertoire *corpus* avec deux sous-répertoires *positive* et *negative* qui contiennent chacun une dizaine de messages classés comme globalement positifs ou négatifs ;
- un répertoire *data* avec :
   - un fichier *SWN.txt* qui est une extraction appauvrie de SentiWordNet 3.0 ;
   - deux fichiers de fréquences *positive.txt* et *negative.txt* enrichis des scores de positivité et de négativité de *SWN.txt* ;
   - un fichier *vocabulary.txt* qui représente le lexique global pondéré du corpus d’apprentissage ;
- et enfin un répertoire *test* qui contient la modélisation des deux nouveaux messages à classer.

**Remarque :** Tous les messages ont été générés par ChatGPT afin qu’ils ressemblent à des commentaires écrits par des touristes en visite dans un pays.

### SentiWordNet, une ressource lexicale pour l’analyse de sentiments

[*SentiWordNet 3.0*](https://github.com/aesuli/SentiWordNet) est une base de données lexicale utilisée en analyse de sentiments (*opinion mining*). Elle associe à chaque synset (ensemble de synonymes) de [WordNet 3.0](https://wordnet.princeton.edu/) trois scores numériques représentant les polarités sentimentales : positif, négatif et objectif (neutre).

Chaque score varie entre 0 et 1 et reflète l’**intensité intrinsèque** du sentiment associé au synset, c’est-à-dire une mesure statique qui ne dépend pas du contexte d’utilisation du mot. Autrement dit, ces scores sont attribués au sens précis du mot dans le synset, indépendamment de la phrase ou du document dans lequel il apparaît.

Cette ressource permet d’évaluer automatiquement le sentiment exprimé dans des textes en s’appuyant sur la signification des mots, ce qui ajoute une dimension **sémantique** à l’analyse et facilite ainsi des tâches comme la classification d’avis ou l’analyse d’opinion.

Le fichier *SWN.txt* ne récupère que les scores de positivité et de négativité des couples lemmes/étiquettes trouvés dans les avis.

#### Quelques notions de linguistique

**Lemme :** en linguistique, le lemme désigne la forme canonique d’un mot. En traitement automatique d’un texte, l’opération de lemmatisation intervient après la segmentation en mots. C’est une phase importante qui demande d’être sensibilisé aux questions de désambiguïsation, car un même mot peut avoir plusieurs formes fléchies (pluriel, conjugaison, genre) et plusieurs sens. La lemmatisation consiste à ramener chaque forme fléchie à sa forme de base pour faciliter l’analyse.

**Étiquette en partie du discours (POS tag) :** C’est une annotation qui indique la catégorie grammaticale d’un mot dans une phrase, comme nom, verbe, adjectif, adverbe, etc. Cette étape permet de mieux comprendre le rôle syntaxique de chaque mot et est essentielle pour des tâches telles que la désambiguïsation, la reconnaissance d’entités nommées ou l’analyse syntaxique.

### Un lexique pondéré ?

Le fichier *vocabulary.txt* a été établi à partir des fichiers *negative.txt* et *positive.txt* afin de ressortir une mesure combinée de la polarité lexicale (score de positivité - score de négativité) et de la fréquence d’occurrence.

La colonne *weight* représente une quantification numérique de la polarité du couple (lemme, étiquette) dans le contexte d’apprentissage. Elle a été mesurée avec la formule suivante :

$$
\text{weight} = \text{freq} \times (\text{posScore} - \text{negScore})_+
$$

Où :

- **freq** est la fréquence d’apparition du couple (lemme, étiquette) dans le corpus (positif ou négatif) ;
- **posScore** et **negScore** sont respectivement les scores de positivité et de négativité extraits de SentiWordNet.

Cette formule garantit que seules les polarités positives sont conservées (les valeurs négatives étant ramenées à zéro), ce qui est adapté aux modèles naïfs bayésiens multinomiaux qui nécessitent des poids non négatifs.

## Un classifieur naïf bayésien

L’algorithme de classification naïve bayésienne, largement utilisé en intelligence artificielle, repose sur le théorème de Bayes pour prédire la classe d’une donnée tout en supposant l’indépendance de ses caractéristiques.

### Hypothèse d’indépendance

#### Une hypothèse bien naïve

La taille d’un chat a-t-elle une influence sur la longueur de son appendice caudal ? Le nombre d’heures de sommeil joue-t-il une quelconque importance dans l’état de vigilance d’une personne ? Le nombre d’enfants est-il en relation avec le type de la voiture qu’un foyer possède ?

À raison, on aurait tendance à répondre positivement à toutes ces questions, mais pas un classifieur naïf bayésien. Pour lui, aucune caractéristique n’influence une autre. Bien que cette hypothèse soit irréaliste dans la plupart des cas, elle a prouvé sa solidité en termes de résultats. Qui plus est, elle permet de simplifier les calculs et d’obtenir des modèles légers et rapides, ce qui en fait une méthode particulièrement efficace lorsque la quantité de données est faible ou que les ressources technologiques sont limitées.

#### Application à une tâche de classification

Une tâche de classification a pour objectif d’associer à un objet une classe ($y$) en fonction de caractéristiques ($X$). On peut le traduire en termes de probabilités d’obtenir $y$ sachant $X$ :

$$
P(y \mid X) = \frac{P(X \cap y)}{P(X)} = \frac{P(X \mid y) \cdot P(y)}{P(X)}
$$

Comme $X$ est une matrice de facteurs ($x_1 \,, x_2 \,, \dots \,, x_n$) indépendants les uns des autres, la formule peut s’exprimer avec un produit de conditions indépendantes :

$$
P(y \mid x_1 \,, x_2 \,, \dots \,, x_n) = \frac{P (y) \times \prod_{i=1}^n P(x_{i} \mid y)}{\prod_{i=1}^n P(x_{i})}
$$

Et comme pour toutes les observations du jeu de données le dénominateur est constant, on peut le supprimer en inférant une notion de proportionnalité :

$$
P(y \mid x_{1} \,, x_{2} \,, \dots \,, x_n) \propto P (y) \times \prod_{i=1}^n P(x_{i} \mid y)
$$

La fonction prédictive est appliquée à l’ensemble des classes possibles et le maximum a posteriori (MAP) sert ensuite à élire la classe la plus plausible :

$$
\hat{y} = \operatorname*{argmax}_{y \in \{C_1, C_2, \dots, C_K\}} P(y) \cdot \prod_{i=1}^n P(x_i \mid y)
$$

**Remarque :** Si les facteurs n’avaient pas été jugés indépendants, la **règle du produit** qui exprime des probabilités conjointes sous forme de produits de probabilités conditionnelles entraînerait des calculs bien plus complexes :  

$$
P(x_1 \,, x_2 \,, \dots \,, x_n \mid y) = P(x_1 \mid y) \times P(x_2 \mid x_1 \cap y) \times P(x_3 \mid x_1 \cap x_2 \cap y) \times \dots \times P(x_n \mid x_1 \cap x_2 \cap \dots \cap x_{n-1} \cap, y)
$$

### Les types de classifieurs

On distingue trois catégories de classifieurs :

- **Le naïf bayésien multinomial :** utilisé principalement pour les données discrètes, comme la classification de texte où l’on comptabilise les occurrences de mots.
- **Le naïf bayésien gaussien :** utilisé lorsque les données sont continues et supposent une distribution normale pour chaque caractéristique au sein de chaque classe. Si ce n’est pas le cas, une transformation peut être appliquée, voire un regroupement en classes, mais c’est au prix d’une perte sans doute conséquente d’information.
- **Le naïf bayésien de Bernoulli :** adapté aux données binaires comme lorsque l’on signale la présence ou l’absence d’un mot plutôt que de compter ses occurrences.

En pratique, il est rare qu’un jeu de données ne présente qu’un seul type de variables, aussi on adopte plutôt une approche hybride où chaque variable est modélisée conformément à sa distribution.

## Étape 1 : Calculer la probabilité conditionnelle de chaque lemme selon la classe

Nous souhaitons connaître à présent la probabilité d’apparition de chaque lemme selon le contexte. Il s’agit de calculer pour un lemme :

$$
P(w_i \mid y) = \frac{F(w_i, y)}{F(y)}
$$

Où :

- $F(w_i \,, y)$ est la fréquence du mot $w_i$ dans la classe envisagée ;
- $F(y)$ est la fréquence totale de la classe envisagée dans l’ensemble des données.

Par exemple, considérons que le mot *be* en tant que verbe apparaît dans la classe positive 20 fois sur 130 :

$$
P(\text{be} \mid +) = \frac{F(\text{be}, +)}{F(+)} = \frac{20}{130} = 0,1538
$$

**Attention !** Comme notre objectif est d’entraîner un classifieur bayésien naïf, nous devons prévoir dans notre modèle le cas où une probabilité serait nulle dans l’une des deux classes, ce qui est d’ailleurs assez souvent le cas dans nos données. Pour éviter l’écueil des probabilités nulles, ajoutez un lissage de Laplace aux probabilités. la formule devient :

$$
P(w_i \mid y) = \frac{F(w_i, y) + 1}{F(y) + V}
$$

Avec $V$ pour la taille du vocabulaire. Pour notre exemple, en donnant $V = 180$, nous obtenons :

$$
P(\text{be} \mid +) = \frac{F(\text{be}, +) + 1}{F(+) + V} = \frac{21}{130 + 180} = 0,0677
$$

Pour réaliser cette première étape, nous avons donc besoin de calculer $F(+)$, $F(-)$ et $V$.

**Pourquoi un lissage ?** Le modèle probabiliste que nous construisons repose sur la multiplication de probabilités. Si un lemme n’apparaît jamais dans un contexte, sa probabilité d’apparition dans ce contexte sera de 0. De là, même si tous les autres lemmes avaient une probabilité de 1 (ils apparaissent systématiquement dans le contexte), on aurait à résoudre $1 \times 1 \dots \times 1 \times 0$ ce qui reviendrait à un résultat improbable de 0.

### Calculer la fréquence de chaque classe

Le code ci-dessous construit un *data frame* à partir du fichier *positive.txt* et effectue la somme des quantités dans la colonne *frequencies* :

In [ ]:
import pandas as pd

# Define column names
cols = ['frequencies', 'lemma', 'tag', 'posScore', 'negScore']

# Load the file with tab separator
df_pos = pd.read_csv('data/positive.txt', sep='\t', header=None, names=cols)

# Calculate the sum of all values in the 'frequencies' column
pos_sum = df_pos['frequencies'].sum()

Confirmez, en affichant `pos_sum` qu’il y a bien 359 lemmes dans les avis positifs.

**Astuce :** Vous pouvez afficher le *data frame* avec les fonctions `display(df_pos)` ou `print(df_pos)`, ou simplement avec l’instruction `df_pos`.

In [ ]:
# your code here

Essayez à présent de créer une variable `neg_sum` pour les fréquences totales des lemmes en contexte négatif :

In [ ]:
# your code here

### Calculer la taille du vocabulaire

dditionner simplement les variables `pos_sum` et `neg_sum` n’aurait aucun sens : certains lemmes apparaissent en effet dans les deux contextes.

Commencez par fusionner les deux *data frames* en un seul :

In [ ]:
df = pd.concat([df_pos, df_neg], ignore_index=True)

Puis effectuez un dédoublonnage afin de ne conserver que les couples *lemma*/*tag* uniques :

In [ ]:
df = df.groupby(['lemma', 'tag'], as_index=False).agg({
    'frequencies': 'sum',
})

À présent, vous devriez être en mesure d’enregistrer dans une variable `V` la taille du vocabulaire et trouver un total de 680 lemmes :

In [ ]:
# your code here

### Déterminer les probabilités conditionnelles

Maintenant que vous êtes en possession de tous les termes nécessaires, vous allez ajouter dans les *data frames* correspondants les probabilités d’apparition de chaque lemme.

**Rappel :** La formule conditionnelle que nous cherchons à implémenter est :

$$
P(w_i \mid y) = \frac{F(w_i, y) + 1}{F(y) + V}
$$

Intégrez en premier lieu la fonction de calcul des probabilités :

In [ ]:
# Define the probability function with a Laplacian smoothing
def P(frequencies, F, V):
    return (frequencies + 1) / (F + V)

L’instruction ci-dessous implémente la fonction pour les lemmes en contexte positif :

In [ ]:
df_pos['frequencies'].apply(lambda f: P(f, pos_sum, V))

Assignez le résultat à une variable `ppos` et répétez l’opération pour disposer également d’une variable `pneg` pour les lemmes en contexte négatif :

In [ ]:
# your code here

Vous pouvez enfin rajouter une colonne à `df_pos` en exécutant l’instruction :

In [ ]:
df_pos['probabilities'] = ppos

Faites de même pour `df_neg` !

In [ ]:
# your code here

## Étape 2 : Pondérer les fréquences

Le modèle est déjà bien avancé mais, à ce stade, nous avons calculé des probabilités sans tenir compte des indications sémantiques de *SWN* à travers les coefficients *posScore* et *negScore*.

Comment faire pour les inclure ? Ces scores sont conçus pour refléter la polarité intrinsèque d’un mot indépendamment de tout contexte. En conséquence, appliquer directement la pondération sur les fréquences permet de mieux capturer l’importance de chaque mot dans la classification. Une autre solution aurait été de pondérer les probabilités déjà ajustées par le lissage, mais au prix d’un risque de distorsion : des mots fortement pondérés mais peu probables verraient leur importance relevée artificiellement.

### Quelle justification à la pondération ?

Le poids agit alors comme un facteur de confiance ou **d’intensité de polarité**. Par exemple, l’adjectif *excellent* véhicule plus de polarité que *good*. Pondérer $P(w_i \mid y)$ revient ainsi à affirmer que cette probabilité est plus fiable si la polarité est forte. Un autre effet positif de la pondération est qu’elle améliore souvent les classifieurs naïfs en évitant que les mots fréquents mais sémantiquement neutres dominent. La vraisemblance des mots est ainsi corrigée par un signal de polarité empirique très utile en classification.

### La méthode

Le fichier *vocabulary.txt* expose déjà une colonne *weight* qui consigne le poids à ajouter. Nous pouvons maintenant adapter notre modèle :

$$
P(w_i \mid y) = \frac{F(w_i, y) + 1}{F(y) + V} \times \text{weight}_i
$$

Commencez par instancier la variable `df_vocab` avec les données du fichier *vocabulary.txt* :

In [ ]:
# your code here

Ajoutez ensuite au *data frame* `df_pos` l’information sur le poids du lemme :

In [ ]:
# Filter df_vocab on label == 'positive'
df_vocab_pos = df_vocab[df_vocab['label'] == 'positive']

# Merge on 'lemma' & 'tag' columns
df_pos = df_pos.merge(
  df_vocab_pos[['lemma', 'tag', 'weight']],
  on=['lemma', 'tag'],
  how='left'
)

Vous pouvez ajouter à présent une colonne *p_weighted* à `df_pos` pour enregistrer la pondération :

In [ ]:
# your code here
#df_pos['probabilities'] * df_pos['weight'].fillna(1)

Répétez les opérations pour `df_neg` :

In [ ]:
# your code here

### Note sur la cohérence de la probabilité

Pour garder la cohérence probabiliste et obtenir une vraie distribution de probabilité, il faudrait normaliser en pondérant ainsi :

$$
P(w_i \mid y) = \frac{(F(w_i, y) + 1) \cdot \text{weight}_i}{\sum\limits_j (F(w_j, y) + 1) \cdot \text{weight}_j}
$$

Cette formule garantit que la somme des probabilités sur tous les mots $w_j$ est égale à 1, ce qui est essentiel pour une interprétation probabiliste rigoureuse. Cependant, dans de nombreuses applications pratiques, notamment en classification naïve bayésienne, on cherche simplement à comparer les scores relatifs des classes via un $\arg\max$. Dans ce contexte, la normalisation peut être omise car elle ne change pas l’ordre relatif des scores. On peut alors utiliser une version non normalisée comme :

$$
P'(w_i \mid y) = \frac{F(w_i, y) + 1}{F(y) + V} \times \text{weight}_i
$$

Cette pondération simplifie les calculs sans perdre en performance pour la décision finale, une flexibilité somme toute courante en apprentissage automatique et en traitement du langage naturel.

## Étape 3 : Calculer la probabilité a priori des classes

Il s’agit à présent de calculer $P(+)$ et $P(-)$ qui interviennent dans la formule de Bayes. De nombreuses approches sont envisageables. Nous en retenons trois qui définissent notre connaissance *a priori*.

### L’approche naïve

Une première approche serait de faire le rapport entre le nombre de textes dans chaque contexte et leur nombre total, ce qui, comme le corpus d’apprentissage est équilibré (10 textes dans chaque classe), reviendrait à ne favoriser aucune classe a priori :

$$
\begin{aligned}
P(+) &= \frac{F(+)}{F(+) + F(-)} = \frac{10}{20} = 0,5\\
P(-) &= \frac{F(-)}{F(+) + F(-)} = \frac{10}{20} = 0,5
\end{aligned}
$$

Si vous validez cette approche, vous pouvez exécuter la cellule suivante :

In [ ]:
prior_pos = 0.5
prior_neg = 0.5

### L’approche fréquentiste brute

Une autre approche consiste à estimer les probabilités de chaque classe en tenant compte de la distribution des occurrences de mots dans chacune d’elles. 

Si l’on admet qu’il y a 130 occurrences de mots en contexte positif et 150 en contexte négatif, alors :

$$
\begin{aligned}
P(+) &= \frac{\sum F(w_{+})}{\sum F(w_{+}) + \sum F(w_{-})} = \frac{130}{280} \approx 0{,}4643 \\
P(-) &= \frac{\sum F(w_{-})}{\sum F(w_{+}) + \sum F(w_{-})} = \frac{150}{280} \approx 0{,}5357
\end{aligned}
$$

Cette estimation reflète la fréquence totale des mots observés dans chaque classe, et permet d’ajuster la probabilité a priori selon la distribution lexicale effective dans les corpus.

Si vous retenez cette approche, et sachant que vous disposez déjà de la somme des lemmes en contexte positifs et négatifs, calculez `prior_pos` et `prior_neg` :

In [ ]:
# your code here

### L’approche cohérente

Et pour rester cohérent·es avec la pondération de chaque mot par leur score de sentiment, vous pourriez l’inclure aussi dans la formule :

$$
\begin{aligned}
P(+) &= \frac{\sum_i (F(w_{+,i}) \times \text{weight}_i)}{\sum_i (F(w_{+,i}) \times \text{weight}_i) + \sum_j (F(w_{-,j}) \times \text{weight}_j)}\\
P(-) &= \frac{\sum_j (F(w_{-,j}) \times \text{weight}_j)}{\sum_i (F(w_{+,i}) \times \text{weight}_i) + \sum_j (F(w_{-,j}) \times \text{weight}_j)}
\end{aligned}
$$

Cette version pondérée permet de prendre en compte non seulement la fréquence des mots, mais aussi leur importance lexicale via leur poids associé.

Pour la calculer, nous avons besoin de connaître au préalable la somme des poids en contextes positifs et négatifs.

**Attention !** Afin de bien prendre en considération la contribution d’un lemme aux probabilités de sa classe, nous veillons à multiplier son poids par le nombre de ses apparitions.

In [ ]:
pos_weighted = (df_pos['frequencies'] * df_pos['weight']).sum()
neg_weighted = (df_neg['frequencies'] * df_neg['weight']).sum()

Il ne reste plus qu’à calculer `prior_pos` et `prior_neg` :

In [ ]:
# your code here

## Étape 4 : Appliquer le modèle

### Les commentaires à évaluer

Vous recevez deux nouveaux commentaires :

>« The splendid medieval castle stands beautifully preserved on top of a natural landscape, reflecting centuries of fascinating history. The knowledgeable guide made our tour truly unforgettable with her charming storytelling and helpful explanations about the architecture. The exquisite surroundings and breathtaking views from the walls created such a profound sense of awe that I would highly recommend this experience to any history lover. »

>« The museum's architecture is interesting and hard to ignore, with clear natural lighting that makes some exhibits seem challenging to appreciate. The historical significance of this place is well-preserved, though walking through the quiet halls feels surprisingly exhausting after a while. While the collections are worth seeing for their profound cultural importance, the overall experience might make visitors feel comfortable or mediocre depending on their expectations. »

Dans le répertoire *test*, les fichiers *message_1.txt* et *message_2.txt* contiennent la liste des lemmes avec les scores de positivité et de négativité. Certaines particularités sont à observer :

- Certains lemmes des messages ne figurent pas dans les données d’entraînement, aussi les deux scores sont à 0. On pourra les ignorer par la suite car ils n’apporteront rien au calcul.
- D‘autres lemmes n’apparaissent que dans un seul contexte, aussi avons-nous appliqué une règle pour calculer les scores par défaut : $P(w \mid y) = \frac{0 + 1}{F + V}$. Encore une fois, comme vous allez calculer les probabilités dans chaque contexte, une valeur de 0 provoquerait immédiatement la disqualification de la classe en question.

### Affinage du modèle

La formule de prédiction retenue par application du théorème de Bayes était :

$$
\hat{y} = \operatorname*{argmax}_{y \in \{C_1, C_2, \dots, C_K\}} P(y) \cdot \prod_{i=1}^n P(x_i \mid y)
$$

En pratique, face aux problèmes soulevés par les petites probabilités, on calcule le maximum de vraisemblance d’une classe grâce au logarithme :

$$
\hat{y} = \operatorname*{argmax}_{y \in \{C_1, C_2, \dots, C_K\}} \log \left( P(y) \times \prod_{i=1}^n P(x_i \mid y) \right)
$$

En utilisant la propriété du logarithme (le produit devient une somme) :

$$
\hat{y} = \operatorname*{argmax}_{y \in \{C_1, C_2, \dots, C_K\}} \left[ \log P(y) + \sum_{i=1}^n \log P(x_i \mid y) \right]
$$

Et, afin de conserver la contribution totale d’un lemme à la prédiction, nous multiplions sa log-probabilité par sa fréquence, représentée par $F(x_i)$ dans la formule :

$$
\hat{y} = \operatorname*{argmax}_{y \in \{C_1, C_2, \dots, C_K\}} \left[ \log P(y) + \sum_{i=1}^n F(x_i) \cdot \log P(x_i \mid y) \right]
$$

#### Pourquoi les petites probabilités posent-elles problème ?

Quand on multiplie des probabilités (toutes entre 0 et 1), on obtient rapidement des nombres minuscules :

$$
0,1 \times 0,2 \times 0,05 \times 0,3 = 0,0003
$$

Avec 50 probabilités de 0,1 chacune, on obtient par exemple : $0,1^{50} \approx 10^{-50}$

Les ordinateurs ne peuvent pas représenter des nombres aussi petits avec précision, ce qui provoque des erreurs d’arrondi ou des *underflows* (le nombre devient 0).

Une manière simple de se rendre compte de la difficulté de la tâche est de demander aux ordinateurs de calculer une exponentiation basique :

$$
0,1^2 = 0,1 \times 0,1 = 0,01
$$

Évaluez le code ci-dessous pour vous convaincre qu’on n’envoie pas Thomas Pesquet sur la lune avec Python :

In [ ]:
0.1 ** 2

Dans notre cas d’étude, $P(w_i \mid y)$ sera en effet souvent très proche de zéro aussi le produit de toutes les probabilités aboutira à une valeur si proche de zéro que l’ordinateur aura du mal à la manipuler. Le logarithme présente alors l’avantage de transformer le produit en somme, ce qui facilite les calculs et est en plus numériquement plus stable.

#### Note sur le logarithme

En termes simples, le logarithme répond à la question : « À quelle puissance dois-je élever ma base pour obtenir ce nombre ? »

Par exemple, dans un système binaire, on établit que $2^3 = 8$. De là, on sait que le logarithme en base 2 de 8 est 3. On notera : $\log_2(8) = 3$.

En fonction du contexte, on est amené·es à manipuler des bases différentes. En plus du logarithme binaire, citons le logarithme décimal et le logarithme naturel qui utilise la base $e \approx 2,718$.

L’une des propriétés intéressantes du logarithme est qu’il permet de transformer un produit en somme grâce à la relation fondamentale suivante :

$$
\log(a \times b) = \log(a) + \log(b)
$$

En *machine learning*, on utilise couramment le logarithme naturel pour trois raisons principales :

1. **Simplification des dérivées :** $\frac{d}{dx}[\ln(x)] = \frac{1}{x}$ ;
2. **Optimisation :** Les algorithmes de descente de gradients convergent mieux ;
3. **Entropie :** Les mesures d’information utilisent naturellement $\ln$.

### Résoudre la tâche de prédiction

Il est maintenant temps de passer à la prédiction. Votre modèle étant entraîné, vous disposez de tous les éléments nécessaires pour estimer la classe $\hat y$ d’un nouvel exemple, à l’aide de la règle du **maximum a posteriori** (MAP), propre au classifieur bayésien naïf :

$$
\hat{y} = \operatorname*{argmax}_{y \in \{C_1, C_2, \dots, C_K\}} \left[ \log P(y) + \sum_{i=1}^n F(x_i) \cdot \log P(x_i \mid y) \right]
$$

À l’étape précédente, vous avez en effet déterminé $P(y)$ et, précédemment, vous avez calculé $P(x_i \mid y)$ soit toutes les probabilités conditionnelles d’apparition des lemmes en fonction de la classe.

#### Étape 1 : Charger les données

Instanciez une variable `df` qui charge les données du fichier *message1.txt* :

In [ ]:
# your code here

Si vous jetez ensuite un coup d’œil au *data frame* obtenu, vous observerez qu’il contient bien les probabilités conditionnelles des lemmes sachant la classe :

In [ ]:
df.head()

#### Étape 2 : Filtrer les probabilités nulles

En se concentrant dans un premier temps sur l’évaluation de la classe positive, le code suivant filtre les entrées dont la variable *p_pos* est nulle :

In [ ]:
# Keep only rows with non-zero conditional probability for positive class
df_pos = df[df["p_pos"] > 0.0].copy()

#### Étape 3 : Calculer les log-probabilités pondérés par la fréquence

Commencez par importer la bibliothèque *Numpy* qui donne accès à une fonction qui calcule le logarithme naturel :

In [ ]:
import numpy as np

Ajoutez maintenant à `df_pos` une colonne *log_p_pos* qui contienne le score de log-probabilité :

In [ ]:
df_pos["log_p_pos"] = np.log(df_pos["p_pos"])

À vous de rajouter une colonne *w_log_p_pos* qui contienne le produit entre le score de log-probabilité et le nombre d’apparition du lemme :

In [ ]:
# your code here

Il ne vous reste plus qu’à calculer la fonction de vraisemblance (*log-likelihood*). Instanciez une variable `log_likelihood_pos` qui enregistre la somme de la colonne *w_log_p_pos* :

In [ ]:
# your code here

#### Étape 4 : Calculer le MAP score

En combinant la vraisemblance (*log-likelihood*) et l’a priori (*prior*), vous obtenez le MAP score pour la classe positive :

In [ ]:
# your code here

Répétez toutes les étapes pour obtenir le MAP score de la classe négative :

In [ ]:
# your code here

#### Étape 5 : Calculer le maximum de vraisemblance

En comparant `score_pos` et `score_neg` pour pourrez déterminer si le message en question est plutôt positif ou négatif :

In [ ]:
# your code here

## Discussion

Déçu·es par la prédiction ? Reproduisez toutes les étapes avec le deuxième message, un peu plus nuancé que le premier : vous attendiez-vous à ce résultat ? Quels sont selon vous les facteurs qui peuvent influencer le résultat ?